# Projet 2: Concevez une application au service de la santé publique

## Notebook de nettoyage des données

### chargement des modules Python

In [1]:
import os
import pandas as pd
import numpy as np
import validators # to check url format
import urllib.request # to check if url is reachable
from deep_translator import GoogleTranslator # to translate
import pycountry # to obtain a list of countries
import unidecode 
import pickle
# To activate if necessary
#from joblib import Parallel, delayed 

### Fonctions utiles

#### fonctions d'affichage

In [2]:
def print_random(data, n):
    """
    function to print n random rows in the dataframe
    parameters : data : pd.DataFrame
                 n : int
    returns : None
    """
    i = np.random.randint(0, len(data), size=n)
    print(data.iloc[i])

#### Fonctions pour la variable code

In [3]:
def clean_code(value, verbose=1):
    """
    Function to clean the code of products of Open Food Facts
    return a code of 0 for wrong values
    
    Parameters : value : str or int or float
                    Value to clean
                 verbose : int [0,1], default = 1
                     Verbose level : 0 quiet
                                     1 print modified value
    
    Returns : str  
    """
    val = np.NaN
    done = False
    if type(value) == str :
        if value.isdigit():
            val = value
            done = True
    elif type(value) == float :
        if value.is_integer():
            val = str(int(value))
            done = True
    elif type(value) == int:
        print(value)
        val = str(value)
        done = True
    else :
        print(f"I do not know how to process type {type(value)}")
        
    if verbose == 1 and not done and pd.notnull(value):
        print(f"{value} is not a integer")
    
    return val
    #l_val = len(val)
    #return "0"*(13-l_val)+val

#### fonctions pour les url

In [4]:
def clean_url(url, code, verify_exists=False, verbose=1):
    """
    Function to clean the url of products of Open Food Facts
    Verify is the url is in a good format and exists
    Verify if the code product of the url is the code product 
    return NaN for wrong url
    
    Parameters : url : str
                    Value to clean
                 code : str
                     code of products
                 verbose : int [0,1], default = 1
                     Verbose level : 0 quiet
                                     1 print modified value
    
    Returns : url 
    """
    def url_reachable(url):
        try :
            code = urllib.request.urlopen(url).getcode()
        except :
            code = 0
        return code == 200
    
    def url_code(url) :
        name = url.replace("http://world-fr.openfoodfacts.org/", '')
        code = name.split("/")
        if len(code) > 0:
            return code[1]
        else :
            return "0"
        
    if len(url) != len(code): # shape checking
        raise Exceptions("Error in input shape parameters")      
        
    url_clean = []
    for i in range(0, len(url)):
        val = str(url.iloc[i])
        if validators.url(val) : # good format
            if url_code(val) == code.iloc[i] :
                if verify_exists : 
                    if url_reachable(val) : # url reachable
                        url_clean.append(val)
                    else: 
                        if verbose == 1 :
                            print(f"{val} is not reachable")
                        url_clean.append(np.NaN)
                else :
                    url_clean.append(val)
            else :
                if verbose ==1: 
                    print(f"the code of url {val} is not the product code {code.iloc[i]}")
                url_clean.append(np.NaN)
        else :
            #print(val, pd.notnull(val), type(val))
            if verbose == 1 and (pd.notnull(val) and val != "nan"):
                print(f"{val} not in a good format")
            url_clean.append(np.NaN)
    return url_clean

In [5]:
def clean_url_para(url, code, verify_exists=False):
    """
    same function than clean_url but using joblib.Parallel

    Parameters : url : str
                    Value to clean
                 code : str
                     code of products
                 verbose : int [0,1], default = 1
                     Verbose level : 0 quiet
                                     1 print modified value
    
    Returns : url 
    """
    def do_one(val, code, i, i_tot):
        def url_code(url) :
            name = url.replace("http://world-fr.openfoodfacts.org/", '')
            code = name.split("/")
            if len(code) > 0:
                return code[1]
            else :
                return "0"

        def url_reachable(url):
            try :
                code = urllib.request.urlopen(url).getcode()
                #print(code)
            except :
                code = 0
            if code == 0 :
                try :
                    code = urllib.request.urlopen(url).getcode()
                except :
                    code = 0
            return code == 200
            
        print(f"step = {i}/{i_tot} : {i/i_tot*100:.2f}%", end="\r")
        # To obtain a good result, it is necessary to add these lines
        # I do not know why => investigations...
        try :
            urllib.request.urlopen(val).getcode()
        except :
            pass
        
        if validators.url(val) : # good format
            if url_code(val) == code :
                if verify_exists :
                    if url_reachable(val) : # url reachable
                        #return [i, val]
                        return val
                    else: 
                        #return [i, np.NaN]
                        return np.NaN
                else :
                    #return [i,val]
                    return val
            else :
                #return [i, np.NaN]
                return np.NaN
        else :
            #return [i,np.NaN]
            return np.NaN
        
            
    if len(url) != len(code): # shape checking
        raise Exceptions("Error in input shape parameters")      
    # Checkpoint     
    i = np.arange(0,len(url), round(len(url)/20))
    i[-1] = len(url)
    
    all = []
    for j in range(1, len(i)) :
        result = Parallel(n_jobs=4, verbose=10)(delayed(do_one)(str(url.iloc[i]), code.iloc[i], i, len(url)) for i in range(i[j-1], i[j]))
        all += result
        result = np.array(result)
        with open("list_url_"+str(i[j-1])+'-'+str(i[j])+".pkl", "wb") as ofile :
            pickle.dump(result, ofile)
    all = np.array(all)
    with open("list_url_all.pkl", "wb") as ofile :
        pickle.dump(all, ofile)
    #return all[:,1]
    return all

#### fonctions pour les str

In [7]:
def is_str(value, min_size=0):
    """
    function to check if the value is a string with minimum characters
    parameters : value
                    Value to check
                 min_size : int
                    Minimun number of char
    returns : str if value is a str with enough char
              np.NaN else
    """
    if type(min_size) != int :
        try : 
            min_size = int(min_size)
        except :
            min_size = 0
        print(f"Warning : min_size is not a integer, reset to {min_size}")
    if type(value) == str and len(value) >= min_size :
        return value
    else :
        if pd.notnull(value) :
            print(f"{value} is not correct value => ignored")
        return np.NaN

In [8]:
def str_replace(value, in_motif, out_motif) :
    """
    function to modify str chains to replace in_motif by out_motif
    if the type of value is not str, return numpy.NaN
    
    parameters : value : str
    
    returns : str
    """
    if type(value) == str :
        return value.replace(in_motif, out_motif)
    elif pd.notnull(value) :
        print(f"Type of {value} is {type(value)} : not expected => ignored ")
        return np.NaN
    else :
        return np.NaN

In [10]:
def remove_str(value, remove_value) :
    """
    function to remove value with a specific str
    parameters : value : str
                 remove_value : str
    return str or np.NaN
    """
    if type(value) == str and value == remove_value:
        return np.NaN
    else :
        return value

In [11]:
def remove_underscore_start_end(value):
    """
    function to modify str chains to remove _ at the beginning and the end of the str
    if the type of value is not str, return numpy.NaN
    
    parameters : value : str
    
    returns : str
    """
    if type(value) == str :
        return value.rstrip('_').lstrip('_')
    else :
        return np.NaN
        

In [12]:
def lower_case(value):
    """
    function to modify str chains to lower case
    if the type of value is not str, return numpy.NaN
    
    parameters : value : str
    
    returns : str.lower()
    """
    if type(value) == str :
        return value.lower()
    elif pd.notnull(value) :
        print(f"Type of {value} is {type(value)} : not expected => ignored ")
        return np.NaN
    else : 
        return np.NaN

In [13]:
def remove_accent(value) :
    """
    function to remove accents from str value
    parameters : value str
    return : str 
    """
    if type(value) == str :
        return unidecode.unidecode(value)
    elif pd.notnull(value) :
        print(f"Type of {value} is {type(value)} : not expected => ignored")
        return np.NaN
    else :
        return np.NaN

In [14]:
def str_in(value, motifs, ignore_case = False):
    """
    function to check if a value is in a list of motif
    if yes, return the value,
    else, return NaN
    
    parameters : value : str
                 motifs: list of str
                 ignore_case : bool
                 
    returns : str
    """
    if type(value) == str :
        if ignore_case :
            motifs = list([val.lower() for val in motifs])
            value = value.lower()
        
        if value in motifs :
            return value.lower()
        else : 
            print(f"{value} is not a correct value: ignored")
            return np.NaN
    else :
        return np.NaN

In [15]:
def find_country(sentence, countries) :
    """
    Function to find a country in a sentence
    return the country found or np.NaN
    
    parameters : sentence : str 
                 country : dict 
                     dict with countries name in different languages
    
    returns : str or np.NaN
    """
    if type(sentence) == str :
        # lower case
        #sentence = sentence.lower()
        # remove accent
        #sentence = unidecode.unidecode(sentence)
        
        #looking for all the sentence in country list
        i_country = list([countries[language].index(sentence) 
                    for language in countries if sentence in countries[language] ] )
        
        words = sentence.split(" ")
        ## lower case
        #words = list(map(lower_case, words))
        ## remove accent
        #words = list(map(unidecode.unidecode, words))
        if len(words) > 1 :
            i_country += list([countries[language].index(word) 
                               for word in words for language in countries 
                               if word in countries[language] ] )
        #print(sentence)
        #print(i_country)
        i_country = list(set(i_country))
        #print(i_country)
        if len(i_country) == 1 :
            country = countries["en"][i_country[0]]
            return country
        elif len(i_country) > 1:
            # if more than one contry, return a list of country split by ","
            #print(f"More than one country found in : {sentence}")
            return ','.join([countries["en"][i_country[i]] for i in range(0, len(i_country)) ] )
        elif '-' in sentence :
            country = find_country(str_replace(sentence, '-', ' '), countries)
            return country
        else :
            # try to replace "-" by space
            print(f"No country detected in {sentence}")
            return np.NaN
    else :
        return np.NaN

#### fonctions pour les variables float

In [16]:
def is_float(value) :
    """
    function to verify if a value is a float
    if yes, return the float, else return numpy.NaN

    parameters : value 
    
    returns : float
    """
    try :
        return float(value)
    except ValueError :
        print(f"{value} is not a float")
        return np.NaN

In [17]:
def is_lower(value, target=np.inf) :
    """
    function to verify if the value is lower than the target value
    parameters : value : float 
                 target : float
    returns : value if the value is lower than the target,
              np.NaN if the value is higher
    """
    if pd.notnull(value) :
        if value <= target :
            return value
        else :
            print(f"{value} is higher than the target {target}")
            return np.NaN
    else :
        return np.NaN

In [19]:
def is_higher(value, target=-np.inf) :
    """
    function to verify if the value is higher than the target value
    parameters : value : float 
                 target : float
    returns : value if the value is higher than the target,
              np.NaN if the value is lower
    """
    if pd.notnull(value) :
        if value >= target :
            return value
        else :
            print(f"{value} is lower than the target {target}")
            return np.NaN
    else :
        return np.NaN

#### fonctions pour les variables int

In [20]:
def is_int(value) :
    """
    function to verify if a value is a int
    if yes, return the int, else, return np.NaN
    parameters : value
    return int
    """
    if pd.isnull(value) :
        return np.NaN
    elif type(value) == int :
        return value
    elif type(value) == float :
        if value.is_integer():
            return int(value)
        else :
            print(f"{value} is not an integer, round to {int(round(value))}")
            return int(round(value))
    elif type(value) == str :
        if value.isdigit() :
            return int(value)
        else :
            print(f"{value} is not a interger => ignored ")
            return np.NaN
    else :
        print(f"Unknown format {type(value)} => ignored")
        return np.NaN

#### fonctions pour les variables list

In [21]:
def create_list(value) :
    """
    function to create list from str chains,
    if the type of value is not str, return empty list
    
    parameters : value : str
    
    returns : list
    """
    if type(value) == str :
        return value.split(',')
    elif (type(value) == int or type(value) == float) and pd.notnull(value) :
        print(f"Type of {value} is {type(value)} : not expected => ignored ")
        return []
    else :
        return []

In [22]:
def list2str(value) :
    """
    function to convert a list to a str with the join function
    parameters : value : str
    return : str if the type of value is list, np.NaN else
    """
    if type(value) == list :
        return ",".join(value)
    else :
        return np.NaN

In [23]:
def list_replace(values, in_motif, out_motif) :
    """
    function to modify list of str chains to replace in_motif by out_motif in each items
    if the type of value is not list, return empty list
    
    parameters : values : list
    
    returns : list
    """
    out = []
    if type(values) == list :
        for value in values :
            out.append(str_replace(value, in_motif, out_motif))
    return out

#### fonctions pour la traduction

In [24]:
def translate_list(values, to='en') :
    """
    function to translate all values of a list to the specific language
    
    parameters : value : list of str
                 
    returns : list of str 
    """
    translated = []
    for value in values :
        source_language = 'auto'
        val = value.split(':')
        if len(val[0]) == 2 and val[0] != "xx" : # xx : unknown country
            source_language = val[0]
        else :
            print(f"Unknown language {val}")
        text = ":".join(val[1:]) # remove the language
        if source_language != to :
            print(text, source_language)
            try : 
                translated.append(GoogleTranslator(source=source_language, 
                                                   target=to).translate(text))
            except :
                translated.append(GoogleTranslator(source='auto', 
                                                   target=to).translate(text))
            print(translated[-1])
        else :
            translated.append(text)
    return translated

In [25]:
def create_translate_dict(lvalues, to='en', old=None) :
    """
    To create a dict of values to translate from a list of lists (one list per row)
    parameters : list of lists ( [ [values_1] ... [values_n] ] )
                    each value should have a specific format : language:text
    returns : dict 
                { language: [text] }
    """
    if not old is None and type(old) == dict :
        to_translate = old
    else :
        to_translate = {}
    for row in lvalues :
        for value in row :
            source_language = 'auto'
            val = value.split(':')
            if len(val) == 1 :
                text = val[0]
            elif len(val) == 0 :
                text = ''
            else :
                if len(val[0]) == 2 and val[0] != "xx" : # xx : unknown country
                    source_language = val[0]
                else :
                    print(f"Unknown language {val}")
                text = ":".join(val[1:]) # remove the language
            if source_language != to and len(text) > 1 and not text.isdigit(): 
                if not source_language in to_translate :
                    to_translate[source_language] = []
                if not text in to_translate[source_language] :
                    to_translate[source_language].append(text)
    return to_translate

In [26]:
def translate_dict(to_translate, to='en', old=None) :
    """
    function to translate texts
    
    parameters : to_translate : dict
                    dict[language] = [text]
                 to : target language
                 old : dict 
                     previous translation
    returns : translated : dict
                dict[language] = [translated_text]
    """
    if not old is None and type(old) == dict :
        translated = old
    else :
        translated = {}
    for language in to_translate :
        print(f"Translation of {language} in progress")
        if not language in translated :
            translated[language] = []
        texts = to_translate[language][len(translated[language]) : ]
        if language != 'auto' :
            if len(texts) > 0 :
                try : 
                    translated[language] += GoogleTranslator(source=language, target=to).translate_batch(texts)
                except :
                    print(f"Failed to translate {language}")
                    del to_translate[language][-len(texts):]
        else :
            for word in texts :
                translated[language].append(GoogleTranslator(source=language, target=to).translate(word))
        print("Done")
    return translated

In [27]:
def apply_translation(values, to_translate, translated, to='en'):
    """
    function to apply the translation 
    
    parameters : values : list
                    list of str
                 to_translate : dict
                     dict of text not translated
                 translated : dict
                     dict of text translated
                 to : str
                     target language
    """
    out = []
    for value in values :
        source_language = 'auto'
        val = value.split(':')
        if len(val) == 1 :
            text = val[0]
        elif len(val) == 0 :
            text = ''
        else :
            if len(val[0]) == 2 and val[0] != "xx" : # xx : unknown country
                source_language = val[0]
            text = ":".join(val[1:]) # remove the language
        if len(text) > 1 :
            if source_language != to :
                if source_language in translated and text in to_translate[source_language] :
                    out.append(translated[source_language][to_translate[source_language].index(text)])
                else :
                    print(f"Failed to translate {text} with original language {source_language}")
            else :
                out.append(text)
        else :
            print(f"Value of text is strange : {text} => ignored")
    return out

#### Fonctions de gestion des doublons

In [28]:
def merge_rows(rows):
    """
    function to merge n rows identify as duplicated
    parameters : DataFrame with n rows
    returns : DataFrame with 1 row
    """
    def merge_str(values) :
        #remove null values
        names = list([value for value in values if pd.notnull(value) ])
        if len(names) == 0 :
            return np.NaN
        else :
            return ','.join(names)
        
    def merge_float(values) :
        # remove null
        values = list([value for value in values if pd.notnull(value)])
        if len(values) == 0 :
            return np.NaN
        else :
            return np.mean(values)
        
    def merge_list(values) :
        #print("init values")
        #print(values)
        new_l = []
        for value in values :
            new_l += value
        #print("summed values")
        #print(new_l)
        #print("unique value")
        #print(list(set(new_l)))
        return list(set(new_l)) # remove same values
    
    def merge_grade(values) :
        alphabet = ['a','b','c','d','e']
        i_values = list([alphabet.index(value) for value in values 
                   if pd.notnull(value) ] )
        if len(i_values) == 0 :
            return np.NaN
        else :
            return alphabet[round(np.mean(i_values))] 
        
    accepted_columns = ['code','url','product_name', "categories_tags", "nutrition_grade_fr",
                        'carbon-footprint_100g',"origins_tags", "manufacturing_places_tags"
                       ]
    #new_row = pd.DataFrame(columns=rows.columns)
    new_row = {}
    for col in rows.columns :
        if col in accepted_columns :
            if col == "code" : # all codes should be equal
                new_row['code'] = [rows['code'].iloc[0]]
            elif col == 'url' : # first value not null
                urls = rows[pd.notnull(rows["url"])]
                if len(urls) > 0 :
                    new_row['url'] = [urls.iloc[0]['url']]
                else :
                    new_row["url"] = [np.NaN]
            elif col == "product_name": #product names are concatenate
                #names = [rows.iloc[i]["product_name"] for i in range(0,len(rows)) 
                #         if pd.notnull(rows.iloc[i]["product_name"])]
                #new_row["product_name"] = ",".join(names)
                new_row['product_name'] = [ merge_str(rows["product_name"].values) ]
            elif col == "categories_tags": # Categories are merge in a single list
                new_row['categories_tags'] = [ merge_list(rows["categories_tags"].values) ]
            elif col == "origins_tags":
                new_row['origins_tags'] = [ merge_list(rows["origins_tags"].values) ]
            elif col == "manufacturing_places_tags" :
                new_row['manufacturing_places_tags'] = [merge_list(rows["manufacturing_places_tags"].values)]
            elif col == "nutrition_grade_fr": # return the average of grades
                new_row["nutrition_grade_fr"] = [ merge_grade(rows["nutrition_grade_fr"].values) ]
            elif col == "carbon-footprint_100g":
                new_row["carbon-footprint_100g"] = [ merge_float(rows['carbon-footprint_100g'].values) ]
        else :
            print(f"columns name {col} is not known, the value of the first row used. Please update the function")
            new_row[col] = [rows.iloc[0][col]]
    return pd.DataFrame(new_row)

### Variables globales

In [31]:
# PATH os the original database
DATA_FILE = "fr.openfoodfacts.org.products.csv"

COUNTRIES = {}
# List of countries obtained from the pycountry module
COUNTRIES["en"] = list([str_replace(country.name.lower(), ' ', '-') 
                        for country in pycountry.countries]) 

# Some country have a "," in the name, we keep only the first part
COUNTRIES['en'] = list([ country.split(',')[0] for country in COUNTRIES['en'] ])
#for country in COUNTRIES['en'] :
#    country = country.split(',')
#    if len(country) > 1 :
#        print(country)

# Translate country names in different languages
for language in ['fr', 'es', 'de'] :
    COUNTRIES[language] = GoogleTranslator(source="en", target=language).translate("\n".join(COUNTRIES['en'])).split('\n')
    # Put all letters in lower case
    COUNTRIES[language] = list(map(lower_case, COUNTRIES[language]))
    # remove accent 
    COUNTRIES[language] = list(map(unidecode.unidecode, COUNTRIES[language]))
    
# add abreviation
COUNTRIES["alpha_3"] = list([country.alpha_3.lower() for country in pycountry.countries]) 
# replace GBR by uk, often found in the database
COUNTRIES["alpha_3"][COUNTRIES["alpha_3"].index("gbr")] = 'uk'

# The tranlation in french add sometime "le" or "la" or "l'"
# We remove it
COUNTRIES['fr'] = list([str_replace(country, 'la ', '') for country in COUNTRIES['fr']])
COUNTRIES['fr'] = list([str_replace(country, 'le ', '') for country in COUNTRIES['fr']])
COUNTRIES['fr'] = list([str_replace(country, "l'", '') for country in COUNTRIES['fr']])

# add by hand some country or region often found in the list but not in the country list
for country in [{'en': 'european union', 'fr': 'union europeenne', 'es': 'union europea', 
                 'de': 'europaische union', 'alpha_3': 'ue'},
               {'en': 'england', 'fr': 'angleterre', 'es': 'inglaterra', 'de': 'england', 'alpha_3': 'eng'},
               {'en': 'scotland', 'fr': 'ecosse', 'es': 'escocia', 'de': 'schottland', 'alpha_3': 'sco'},
               {'en': 'czech republic', 'fr': 'republique tcheque', 'es': 'republica checa', 
                'de': 'tschechien', 'alpha_3': 'cze2'}, # it should be better to say it equal to czechia
               {'en': 'russia', 'fr': 'russie', 'es': 'rusia', 'de': 'russland', 
                'alpha_3': 'rus2'} # it should be better to say it equal to russian federation
               ]:
    COUNTRIES['en'].append(country['en'])
    COUNTRIES['fr'].append(country['fr'])
    COUNTRIES['es'].append(country['es'])
    COUNTRIES['de'].append(country['de'])
    COUNTRIES['alpha_3'].append(country['alpha_3'])

# Remove translation errors...
# -Turkey is not dinde in french !
i = COUNTRIES['alpha_3'].index("tur")
COUNTRIES['fr'][i] = "turquie"
   
# Verify if country in the list
i = COUNTRIES['alpha_3'].index("tur")
print(COUNTRIES['en'][i])
print(COUNTRIES['fr'][i])
i = COUNTRIES['alpha_3'].index("cze")
print(COUNTRIES['en'][i])
print(COUNTRIES['fr'][i])
i = COUNTRIES['alpha_3'].index("ltu")
print(COUNTRIES['en'][i])
print(COUNTRIES['fr'][i])
i = COUNTRIES['alpha_3'].index("rus")
print(COUNTRIES['en'][i])
print(COUNTRIES['fr'][i])
i = COUNTRIES['alpha_3'].index("zaf")
print(COUNTRIES['en'][i])
print(COUNTRIES['fr'][i])


turkey
turquie
czechia
tchequie
lithuania
lituanie
russian-federation
federation russe
south-africa
afrique du sud


### Chargement du dataset

In [33]:
if os.path.isfile(DATA_FILE):
    DATA = pd.read_csv(DATA_FILE, sep='\t', low_memory=False)
    size = len(DATA)
    n_variables = len(DATA.columns)
    print(f"Taille de l'échantillon = {size}")
    print(f"Nombre de variables     = {n_variables}")
    print("list of variables :")
    print(sorted(list(DATA.columns)))
else :
    print("Error: " + DATA_FILE + " is not a file")


Taille de l'échantillon = 320772
Nombre de variables     = 162
list of variables :
['additives', 'additives_fr', 'additives_n', 'additives_tags', 'alcohol_100g', 'allergens', 'allergens_fr', 'alpha-linolenic-acid_100g', 'arachidic-acid_100g', 'arachidonic-acid_100g', 'behenic-acid_100g', 'beta-carotene_100g', 'bicarbonate_100g', 'biotin_100g', 'brands', 'brands_tags', 'butyric-acid_100g', 'caffeine_100g', 'calcium_100g', 'capric-acid_100g', 'caproic-acid_100g', 'caprylic-acid_100g', 'carbohydrates_100g', 'carbon-footprint_100g', 'casein_100g', 'categories', 'categories_fr', 'categories_tags', 'cerotic-acid_100g', 'chloride_100g', 'chlorophyl_100g', 'cholesterol_100g', 'chromium_100g', 'cities', 'cities_tags', 'cocoa_100g', 'code', 'collagen-meat-protein-ratio_100g', 'copper_100g', 'countries', 'countries_fr', 'countries_tags', 'created_datetime', 'created_t', 'creator', 'dihomo-gamma-linolenic-acid_100g', 'docosahexaenoic-acid_100g', 'eicosapentaenoic-acid_100g', 'elaidic-acid_100g', '

La base de données comporte un grand nombre d'échantillons et de variables.  
Il est nécessaire de sélectionner uniquement les variables utiles à l'application.
Il est à noter que la page web d'openfoodfacts regroupant les variables n'est pas à jour :  
- certaines variables dans la base de données ne sont pas indiquées  
- à l'invserse, certaines variables sur la page web ne sont pas dans la base de données  

### Sélection des colonnes  
On créé une nouvelle variable avec uniquement les colonnes utiles à l'application après vérification que la colonne existe.  
Les colonnes sélectionnées permettent de :  
- définir le produit : code, product_name,  categories_tags
- donner aux utilisateurs l'ensemble des informations via l'url  
- caractériser son origine : manufacturing_place_tags, origins_tags
- caractériser si il est sein ou non : nutrition_grade_fr
- caractériser son empreinte_carbone : carbon-footprint_100g  
  
L'analyse univariée avec ces variables a montré que trop peu de produits sont complétement renseignés.  
Le choix a été fait de choisir de nouvelles variables avec un taux de remplissage important (testé dans le notebook d'analyse). 
Les nouvelles variables sont :
- additives_n (nombre d'additifs)  
- energy_100 (apport calorifique)  
- fat_100g (proportion de graisse)  
- saturated-fat_100g (proportion de graisses saturées)  
- fruits-vegetables-nuts_100g (proportion de fruits et légumes)  
- sodium_100g (teneur en sel)  
- sugars_100g (teneur en sucre)  

Les variables pnns_groups_1 et pnns_groups_2 catégorisent également les produits. Elles ne sont pas répertoriées sur le site OpenFoodFact, mais semble intéressantes.

In [34]:
# colonnes souhaitées
selected_columns = ['code','url','product_name', "categories_tags", "nutrition_grade_fr",
                    'carbon-footprint_100g',"origins_tags", "manufacturing_places_tags",
                    'additives_n', 'energy_100g', 'fat_100g', 'saturated-fat_100g', 
                    'fruits-vegetables-nuts_100g', 'sodium_100g', 'sugars_100g',
                    'pnns_groups_1', 'pnns_groups_2'
                    ]
    
# Identification des colonnes introuvables
i_remove = []
for i, val in enumerate(selected_columns):
    if val not in DATA.columns:
        print(f"Wrong columns name : {val}")
        i_remove.append(i)
        
# Suppression des colonnes introuvables      
for i in i_remove[::-1]:
    del selected_columns[i]
print("Used columns = ")
print(selected_columns)

#Creation d'un DataFrame vide que l'on va remplir ensuite
DATA_CLEANED = pd.DataFrame(columns=selected_columns)


Used columns = 
['code', 'url', 'product_name', 'categories_tags', 'nutrition_grade_fr', 'carbon-footprint_100g', 'origins_tags', 'manufacturing_places_tags', 'additives_n', 'energy_100g', 'fat_100g', 'saturated-fat_100g', 'fruits-vegetables-nuts_100g', 'sodium_100g', 'sugars_100g', 'pnns_groups_1', 'pnns_groups_2']


### Nettoyage du code produit
Le code doit comporter que des chiffres  
Seul les valeurs aberrantes sont affichées

In [35]:
nullvalue_before = len(DATA[pd.isnull(DATA["code"])])
print(f"Missing value in code before cleaning = {nullvalue_before}, {nullvalue_before/size*100:.2f}%")

Missing value in code before cleaning = 23, 0.01%


In [36]:
DATA_CLEANED["code"] = DATA["code"].apply(clean_code, verbose=1)

In [37]:
nullvalue_cleaned = len(DATA_CLEANED[pd.isnull(DATA_CLEANED["code"])])
print(f"Missing value in code after cleaning = {nullvalue_cleaned}, {nullvalue_cleaned/size*100:.2f}%")
print(f"Number of value removed = {nullvalue_cleaned - nullvalue_before}, {(nullvalue_cleaned - nullvalue_before)/size*100:.2f}%")

Missing value in code after cleaning = 23, 0.01%
Number of value removed = 0, 0.00%


Aucun code produit ne semble aberrant

### Nettoyage des url
L'url doit avoir le bon format  
L'url doit être accessible (en option)   
L'url doit correspondre au bon code produit

In [38]:
nullvalue_before = len(DATA[pd.isnull(DATA["url"])])
print(f"Missing value in code before cleaning = {nullvalue_before}, {nullvalue_before/size*100:.2f}%")

Missing value in code before cleaning = 23, 0.01%


NB : utiliser verify_exists=False pour un test rapide  
La vérification que l'url existe est (très) long,  
la vérification a été réalisée sur un autre pc.  
Le résultat est rechargé ensuite.  
Une fonction de parallélisation a été créer pour gagner du temps.  
Temps nécessaire : ~10h sur 4 procs

#### nettoyage rapide (sans vérifier que l'url existe)

In [39]:
DATA_CLEANED["url"] = clean_url(DATA["url"], DATA_CLEANED["code"], verify_exists=False, verbose=1) #fast checking

#### chargement du dataframe nettoyé avec vérification que l'url existe

In [40]:
df_url_file = "database_cleaned_url_para.csv"
if os.path.isfile(df_url_file) :
    DF_URL = pd.read_csv(df_url_file, sep=";", low_memory=False)
else :
    DF_URL = pd.DataFrame(columns=selected_columns)
if len(DF_URL) != len(DATA_CLEANED) :
    print("Error in the dataframe loaded")
else :
    DATA_CLEANED['url'] = DF_URL['url'] 

In [41]:
nullvalue_cleaned = len(DATA_CLEANED[pd.isnull(DATA_CLEANED["url"])])
print(f"Missing value in code after cleaning = {nullvalue_cleaned}, {nullvalue_cleaned/size*100:.2f}%")
print(f"Number of value removed = {nullvalue_cleaned - nullvalue_before}, {(nullvalue_cleaned - nullvalue_before)/size*100:.2f}%")

Missing value in code after cleaning = 2375, 0.74%
Number of value removed = 2352, 0.73%


Il apparait que des url ne sont pas accessible, il est donc inutile de garder un lien vers une page erronnée.  
Cependant ce nombre est faible et ne représente que 0.73% de la base de données

### Nettoyage des noms des produits
Le nom des produits doit être de type str  
Le nom des produits doit contenir au moins 4 caractères  
Traduire les noms des produits en minuscules  
Remplacer les espaces par des "_" pour faciliter le parcours des noms

In [42]:
nullvalue_before = len(DATA[pd.isnull(DATA["product_name"])])
print(f"Missing value in code before cleaning = {nullvalue_before}, {nullvalue_before/size*100:.2f}%")

Missing value in code before cleaning = 17762, 5.54%


In [43]:
DATA_CLEANED["product_name"] = DATA["product_name"].apply(is_str, min_size=3)

  is not correct value => ignored
Fe is not correct value => ignored
짜왕 is not correct value => ignored
M is not correct value => ignored
H is not correct value => ignored
통깨 is not correct value => ignored
旺仔 is not correct value => ignored
O is not correct value => ignored
ok is not correct value => ignored
你 is not correct value => ignored
a is not correct value => ignored
W5 is not correct value => ignored
알 is not correct value => ignored
bn is not correct value => ignored
K is not correct value => ignored
3d is not correct value => ignored
P is not correct value => ignored
O is not correct value => ignored
Hf is not correct value => ignored
LF is not correct value => ignored
昆布 is not correct value => ignored
曾粉 is not correct value => ignored
Tf is not correct value => ignored
煎茶 is not correct value => ignored
緑茶 is not correct value => ignored
ゆず is not correct value => ignored
月餅 is not correct value => ignored
Bn is not correct value => ignored
BN is not correct value => ign

In [44]:
DATA_CLEANED["product_name"] = DATA_CLEANED["product_name"].apply(str_replace, args=(" ", "_")).apply(lower_case)

In [45]:
print_random(DATA_CLEANED['product_name'], 10)

21044                     brigham's,_ice_cream,_vanilla
148290           fiorello's,_artisan_gelato,_strawberry
64378                                      lemon_pepper
241990    tournesol_allégée_50%_mg_(tartines_&_cuisson)
163870               lemon_pepper_artisan_chicken_jerky
116151          organic_instant_oatmeal,_apple_cinnamon
311639                          turrón_de_coco_al_cacao
254067                                polenta_savoyarde
124899                nature's_promise,_organic_walnuts
11914                rice_&_pasta_mix,_cheddar_broccoli
Name: product_name, dtype: object


Les noms des produits peuvent commencer ou finir par un espace:  
A supprimer

In [46]:
DATA_CLEANED["product_name"] = DATA_CLEANED['product_name'].apply(remove_underscore_start_end)

In [47]:
print_random(DATA_CLEANED['product_name'], 10)

196504                       maxi_croquantes_nature
164374                    sausage_egg_cheese_slider
66665     hobnobs_rolled_oat_&_whole_wheat_biscuits
83871        marcelled_potato_chips,_salt_&_vinegar
95760                             black_bean_&_corn
36632                 all_natural_dried_cranberries
163035             blueberry_merlot_chandoka_spread
153542               jumbo_cashews_sea_salt_roasted
225609                                  à_l'olivier
180214                         bifidus_ananas_(4_x)
Name: product_name, dtype: object


In [48]:
nullvalue_cleaned = len(DATA_CLEANED[pd.isnull(DATA_CLEANED["product_name"])])
print(f"Missing value in code after cleaning = {nullvalue_cleaned}, {nullvalue_cleaned/size*100:.2f}%")
print(f"Number of value removed = {nullvalue_cleaned - nullvalue_before}, {(nullvalue_cleaned - nullvalue_before)/size*100:.2f}%")

Missing value in code after cleaning = 17819, 5.56%
Number of value removed = 57, 0.02%


57 noms de produits ont été considérés aberrantes, soit 0.02%

### nettoyage des catégories
Creation de l'objet list pour faciliter le parcours  
traduction des tag en anglais  
attention la traduction est longue  
Problème : Google accepte un nombre limité de requète par seconde, heure, jour...  
Donc changement de stratégie : créer un dict avec les champs à traduire et effecter la traduction en une seule requète


In [49]:
nullvalue_before = len(DATA[pd.isnull(DATA["categories_tags"])])
print(f"Missing value in code before cleaning = {nullvalue_before}, {nullvalue_before/size*100:.2f}%")

Missing value in code before cleaning = 236383, 73.69%


In [50]:
DATA_CLEANED["categories_tags"] = DATA["categories_tags"].apply(create_list)

In [51]:
print_random(DATA_CLEANED[DATA_CLEANED['categories_tags'].str.len() != 0]['categories_tags'], 10)

253935    [en:beverages, en:alcoholic-beverages, en:beer...
202114    [en:beverages, en:hot-beverages, en:coffees, e...
208425    [en:plant-based-foods-and-beverages, en:plant-...
178600           [en:meats, en:beef, fr:entrecote-de-boeuf]
203741    [en:fresh-foods, en:dairies, en:desserts, en:d...
246272    [en:meats, en:prepared-meats, fr:charcuteries-...
307179    [en:sugary-snacks, en:biscuits-and-cakes, en:b...
93599     [en:plant-based-foods-and-beverages, en:plant-...
264452           [en:dairies, en:cheeses, en:cream-cheeses]
232277    [en:sugary-snacks, en:biscuits-and-cakes, en:b...
Name: categories_tags, dtype: object


In [52]:
nullvalue_cleaned = len(DATA_CLEANED[DATA_CLEANED["categories_tags"].str.len() == 0])
print(f"Missing value in code after cleaning = {nullvalue_cleaned}, {nullvalue_cleaned/size*100:.2f}%")
print(f"Number of value removed = {nullvalue_cleaned - nullvalue_before}, {(nullvalue_cleaned - nullvalue_before)/size*100:.2f}%")

Missing value in code after cleaning = 236383, 73.69%
Number of value removed = 0, 0.00%


#### Chargement du dictionnaire  
Puisque la traduction est longue, un dictionnaire est sauvegardé pour éviter de relancer à chaque fois  

In [53]:
to_translate_file = "dict_to-translate.pkl"
translated_file = "dict_translated.pkl"
if os.path.isfile(to_translate_file) and \
os.path.isfile(translated_file):
    with open(to_translate_file, 'rb') as file :
        to_translate = pickle.load(file)
    with open(translated_file, 'rb') as file :
        translated = pickle.load(file)
else :
    print("No dict saved")
    to_translate = {}
    translated = {}

#### recherche des valeurs à traduire

In [54]:
to_translate = create_translate_dict(DATA_CLEANED["categories_tags"].values, old=to_translate)

Unknown language ['xx', 'attiksgurka']
Unknown language ['xx', 'liszt']
Unknown language ['xx', 'вафли']
Unknown language ['xx', 'вафли']
Unknown language ['xx', 'кисело-мляко']
Unknown language ['xx', 'barre-chocolatee']
Unknown language ['xx', 'fiskekonserves']
Unknown language ['xx', 'helkonserves']
Unknown language ['xx', 'кисело-мляко']
Unknown language ['xx', '煎茶']
Unknown language ['xx', 'スナック菓子']
Unknown language ['xx', 'ポップコーン']
Unknown language ['xx', 'paste']
Unknown language ['xx', 'paste-făinoase']
Unknown language ['xx', 'grøntsag']
Unknown language ['xx', 'confiseries']
Unknown language ['xx', 'snacks-sucres']
Unknown language ['xx', 'brød']
Unknown language ['xx', 'brød']
Unknown language ['xx', 'fiskepalaeg']
Unknown language ['xx', 'kosttilskud']
Unknown language ['xx', 'remoulade']
Unknown language ['xx', 'orange-juice-from-concentrate']
Unknown language ['xx', 'krydderi']
Unknown language ['xx', 'bouillonterning']
Unknown language ['xx', 'krydderi']
Unknown language

#### traduction

In [55]:
translated = translate_dict(to_translate, old=translated)
#print(translated)

Translation of fr in progress
Done
Translation of de in progress
Done
Translation of it in progress
Done
Translation of es in progress
Done
Translation of auto in progress
Done
Translation of pt in progress
Done
Translation of ge in progress
Failed to translate ge
Done
Translation of nl in progress
Done
Translation of ru in progress
Done
Translation of ar in progress
Done
Translation of ja in progress
Done
Translation of ko in progress
Done
Translation of tr in progress
Done
Translation of la in progress
Done
Translation of ro in progress
Done
Translation of el in progress
Done
Translation of pl in progress
Done
Translation of cs in progress
Done
Translation of zh in progress
Failed to translate zh
Done
Translation of fi in progress
Done
Translation of sv in progress
Done
Translation of da in progress
Done
Translation of hu in progress
Done
Translation of he in progress
Failed to translate he
Done
Translation of sk in progress
Done
Translation of th in progress
Done
Translation of vi i

##### Sauvegarde du dictionnaire traduit pour éviter de le recréer

In [56]:
with open("dict_to-translate.pkl", 'wb') as ofile :
    pickle.dump(to_translate, ofile)
with open("dict_translated.pkl", "wb") as ofile :
    pickle.dump(translated, ofile)

#### Application des traductions

In [57]:
DATA_CLEANED["categories_tags"] = DATA_CLEANED["categories_tags"].apply(apply_translation, args=(to_translate, translated))

Value of text is strange : u => ignored
Failed to translate kakheti with original language ge
Failed to translate sauce-de-soja with original language zh
Value of text is strange : p => ignored
Failed to translate tofu-seche with original language zh
Value of text is strange : 茶 => ignored
Value of text is strange : 0 => ignored
Value of text is strange :  => ignored
Value of text is strange :  => ignored
Failed to translate 啤酒 with original language zh
Failed to translate chinese-wines with original language zh
Failed to translate 复原乳 with original language zh
Failed to translate משקאות-קלים with original language he
Failed to translate aliments-d-origine-vegetale with original language he
Failed to translate aliments-et-boissons-a-base-de-vegetaux with original language he
Failed to translate cereales-et-derives with original language he
Failed to translate cereales-et-pommes-de-terre with original language he
Failed to translate cereales-pour-petit-dejeuner with original language he

#### affichage du résultat

In [58]:
print_random(DATA_CLEANED[DATA_CLEANED["categories_tags"].str.len() != 0 ]['categories_tags'], 10)

226986                       [pie-dough, shortcrust pastry]
195265    [plant-based-foods-and-beverages, plant-based-...
226935                       [pie-dough, shortcrust pastry]
300554    [plant-based-foods-and-beverages, plant-based-...
279659          [groceries, sauces, tomato-sauces, ketchup]
204383    [beverages, alcoholic-beverages, wines, french...
252078    [plant-based-foods-and-beverages, plant-based-...
215406    [plant-based-foods-and-beverages, plant-based-...
265803    [farming-products, bird-eggs, eggs, free-range...
212344    [canned-foods, seafood, fishes, canned-fishes,...
Name: categories_tags, dtype: object


In [59]:
nullvalue_cleaned = len(DATA_CLEANED[DATA_CLEANED["categories_tags"].str.len() == 0])
print(f"Missing value in code after cleaning = {nullvalue_cleaned}, {nullvalue_cleaned/size*100:.2f}%")
print(f"Number of value removed = {nullvalue_cleaned - nullvalue_before}, {(nullvalue_cleaned - nullvalue_before)/size*100:.2f}%")

Missing value in code after cleaning = 236388, 73.69%
Number of value removed = 5, 0.00%


il y a pas une uniformité dans les espaces  
parfois " ", parfois '-'  
choix de mettre que des '-'
Vérifier qu'un tag ne se retrouve pas deux fois dans la même liste

In [60]:
DATA_CLEANED["categories_tags"] = DATA_CLEANED["categories_tags"].apply(list_replace, args=(" ", "-"))


In [61]:
nullvalue_cleaned = len(DATA_CLEANED[DATA_CLEANED["categories_tags"].str.len() == 0])
print(f"Missing value in code after cleaning = {nullvalue_cleaned}, {nullvalue_cleaned/size*100:.2f}%")
print(f"Number of value removed = {nullvalue_cleaned - nullvalue_before}, {(nullvalue_cleaned - nullvalue_before)/size*100:.2f}%")

Missing value in code after cleaning = 236388, 73.69%
Number of value removed = 5, 0.00%


In [62]:
DATA_CLEANED["categories_tags"] = DATA_CLEANED["categories_tags"].apply(set).apply(list)

In [63]:
print_random(DATA_CLEANED[DATA_CLEANED["categories_tags"].str.len() != 0]["categories_tags"], 10)

111684    [farming-products, sweeteners, breakfasts, hon...
311885                        [croque-monsieur, sandwiches]
271446    [carbonated-drinks, carbonated-waters, non-sug...
265864                        [hams, prepared-meats, meats]
253536    [sugary-snacks, biscuits-and-cakes, Breton-pan...
235835    [artificially-sweetened-beverages, iced-teas, ...
219348    [frozen-foods, meals, frozen-ready-made-meals,...
248166    [fruits-and-vegetables-based-foods, plant-base...
204134    [block-of-foie-gras, block-of-foie-gras-with-p...
278109    [fruits-and-vegetables-based-foods, meals, ref...
Name: categories_tags, dtype: object


In [64]:
nullvalue_cleaned = len(DATA_CLEANED[DATA_CLEANED["categories_tags"].str.len() == 0])
print(f"Missing value in code after cleaning = {nullvalue_cleaned}, {nullvalue_cleaned/size*100:.2f}%")
print(f"Number of value removed = {nullvalue_cleaned - nullvalue_before}, {(nullvalue_cleaned - nullvalue_before)/size*100:.2f}%")

Missing value in code after cleaning = 236388, 73.69%
Number of value removed = 5, 0.00%


Uniquement 5 lignes n'ont plus de catégories, cela signifie que lorsque des valeurs aberrantes ont été supprimées,  
il existait une autre catégorie pour ce produit.

### Nettoyage des lieux de productions et transformations
Les lieux de productions doivent être du type str  
Pour faciliter, les valeurs doivent contenir le nom d'un pays  
Sinon elles sont considérées comme aberrantes puisqu'elles ne peuvent pas être traitées automatiquement  
Le résultats sont stockés dans des listes
  
Remplacer les virgules par des " " pour identifier facilement les mots  
Recherche d'un nom de pays parmis les mots  
Il faut supprimer les accents et la majuscule pour ne pas fausser la comparaison

In [65]:
nullvalue_before = len(DATA[pd.isnull(DATA["manufacturing_places_tags"])])
print(f"Missing value in code before cleaning = {nullvalue_before}, {nullvalue_before/size*100:.2f}%")

Missing value in code before cleaning = 284277, 88.62%


In [66]:
DATA_CLEANED["manufacturing_places_tags"] = DATA['manufacturing_places_tags'].apply(is_str)
DATA_CLEANED["manufacturing_places_tags"] = DATA_CLEANED['manufacturing_places_tags'].apply(str_replace, args=(",", " "))
DATA_CLEANED["manufacturing_places_tags"] = DATA_CLEANED["manufacturing_places_tags"].apply(lower_case)
DATA_CLEANED["manufacturing_places_tags"] = DATA_CLEANED["manufacturing_places_tags"].apply(remove_accent)
DATA_CLEANED["manufacturing_places_tags"] = DATA_CLEANED["manufacturing_places_tags"].apply(find_country, args=(COUNTRIES,))
DATA_CLEANED["manufacturing_places_tags"] = DATA_CLEANED["manufacturing_places_tags"].apply(create_list)

No country detected in brossard quebec
No country detected in brossard quebec
No country detected in brossard quebec
No country detected in brossard quebec
No country detected in brossard quebec
No country detected in brossard quebec
No country detected in topco
No country detected in norwalk ct
No country detected in u s a
No country detected in californie
No country detected in u s a
No country detected in united states if america
No country detected in etats unis d amerique
No country detected in marlborough massachusetts
No country detected in u s a
No country detected in us california
No country detected in quetigny
No country detected in tyskland
No country detected in blainville quebec
No country detected in dieppe n b
No country detected in more than one country
No country detected in st donat quebec
No country detected in boisbriand quebec
No country detected in ancaster ontario
No country detected in montreal quebec
No country detected in levis quebec
No country detected in d

No country detected in ibi alicante espanha
No country detected in alemanha
No country detected in la mirande 66240 saint esteve
No country detected in wachau
No country detected in holland
No country detected in ul iuzhnaia sheremet'evskii mkr g dolgoprudnyi moskovskaia oblast' rossiia 141720
No country detected in alemanha
No country detected in 69400 limas
No country detected in lille
No country detected in 47110
No country detected in jacques ijs nv nijverheidsstraat 54 b 2260 westerlo
No country detected in phillipines
No country detected in cote d ivoire
No country detected in delpierre mer et tradition 62360 hesdin l abbe
No country detected in hesdin l abbe
No country detected in la celle sous gouzon 23230
No country detected in xirivella valencia
No country detected in 56700 kervignac
No country detected in e u
No country detected in 47500 montayral
No country detected in delpierre mer et tradition 62360 hesdin l abbe
No country detected in hollande
No country detected in gran

No country detected in mouilleron en pareds vendee
No country detected in mont ventoux
No country detected in provence
No country detected in fabrique a plouedern bretagne
No country detected in provence
No country detected in normandie
No country detected in montsecret
No country detected in pontivy
No country detected in mornant
No country detected in mornant
No country detected in vitenam
No country detected in bordeaxu
No country detected in sud ouest virazeil 47200
No country detected in normandie
No country detected in 31220 martres tolosane
No country detected in val de loire
No country detected in pyrenees
No country detected in bretagne cotes d armor
No country detected in corse
No country detected in corse
No country detected in picardie
No country detected in thierache
No country detected in plescop morbihan
No country detected in alsace
No country detected in moulin meckert diemer krautwiller
No country detected in rungis
No country detected in saint emilion gironde
No coun

No country detected in tikhoretsk krasnodarskii krai rossiia
No country detected in kurskaia oblast' rossiia
No country detected in sankt peterburg rossiia
No country detected in severnaia promzona g labinsk krasnodarskii krai rf
No country detected in saransk mordoviia rossiia
No country detected in pruzhany brestskaia oblast' belarus'
No country detected in pos sady pridon'ia gorodishchenskii raion volgogradskaia oblast' rossiia
No country detected in rossiia volgogradskaia oblast'
No country detected in gagarinskii raion smolenskaia oblast' rossiia p rodomanogo
No country detected in shchiolkovo moskovskaia oblast' rossiia
No country detected in latviia
No country detected in 115114 rossiia g moskva 2 oi paveletskii proezd
No country detected in moskva rossiia khlebozavod No. 18
No country detected in oao cherkizovo moskva rossiia ul bol'shaia cherkizovskaia
No country detected in moskva rossiia
No country detected in khlebozavod No.  18 moskva rossiia
No country detected in g moskv

No country detected in barcelona
No country detected in de th 601 eg
No country detected in alicante
No country detected in madrid
No country detected in madrid
No country detected in sierra de granda 33199 siero asturias
No country detected in barcelona
No country detected in barcelona
No country detected in palencia espanha
No country detected in ibercacao s a u
No country detected in valencia
No country detected in utebo saragoca espanha
No country detected in madrid
No country detected in murcia
No country detected in autol la rioja
No country detected in girona
No country detected in albacete
No country detected in madrid
No country detected in valencia
No country detected in valencia
No country detected in cartagena
No country detected in valencia
No country detected in valencia
No country detected in madrid
No country detected in granollers barcelona
No country detected in barcelona espanha
No country detected in barcelona
No country detected in barcelona
No country detected in 

In [67]:
print_random(DATA_CLEANED[DATA_CLEANED["manufacturing_places_tags"].str.len() != 0 ]["manufacturing_places_tags"], 10)


308767      [spain]
245020     [france]
238078     [france]
184142    [germany]
280086    [belgium]
219845     [france]
195882     [france]
308964      [spain]
311890      [spain]
101026      [spain]
Name: manufacturing_places_tags, dtype: object


In [68]:
nullvalue_cleaned = len(DATA_CLEANED[DATA_CLEANED["manufacturing_places_tags"].str.len() == 0])
print(f"Missing value in code after cleaning = {nullvalue_cleaned}, {nullvalue_cleaned/size*100:.2f}%")
print(f"Number of value removed = {nullvalue_cleaned - nullvalue_before}, {(nullvalue_cleaned - nullvalue_before)/size*100:.2f}%")

Missing value in code after cleaning = 286125, 89.20%
Number of value removed = 1848, 0.58%


la majorité des lignes ont été traitées  
Il manque cependant 1848 lignes,  
Cela est principalement dû à la présence du nom de ville ou de région difficile à traiter  
Puisque cela ne représente que 5% du nombre de lignes initialement remplies et 0.6% du nombre total de lignes,  
j'ai décidé de ne pas aller plus loin

### Netoyage de l'origine des produits
Exactement la même démarche que pour les lieux de transformations

In [69]:
nullvalue_before = len(DATA[pd.isnull(DATA["origins_tags"])])
print(f"Missing value in code before cleaning = {nullvalue_before}, {nullvalue_before/size*100:.2f}%")

Missing value in code before cleaning = 298619, 93.09%


In [70]:
DATA_CLEANED["origins_tags"] = DATA['origins_tags'].apply(is_str)
DATA_CLEANED["origins_tags"] = DATA_CLEANED['origins_tags'].apply(str_replace, args=(",", " "))
DATA_CLEANED["origins_tags"] = DATA_CLEANED["origins_tags"].apply(lower_case)
DATA_CLEANED["origins_tags"] = DATA_CLEANED["origins_tags"].apply(remove_accent)
DATA_CLEANED["origins_tags"] = DATA_CLEANED["origins_tags"].apply(find_country, args=(COUNTRIES,))
DATA_CLEANED["origins_tags"] = DATA_CLEANED["origins_tags"].apply(create_list)

No country detected in quebec
No country detected in quebec
No country detected in quebec
No country detected in quebec
No country detected in quebec
No country detected in quebec
No country detected in grande bretagne
No country detected in saudi
No country detected in several countries
No country detected in more than one country
No country detected in california
No country detected in united states of america
No country detected in arden hilss minnesota
No country detected in hackettstown nj
No country detected in californie
No country detected in massachusetts
No country detected in us
No country detected in almendras california estados unidos
No country detected in almendras california estados unidos
No country detected in north america
No country detected in europe
No country detected in ivory coast
No country detected in porc du royaume uni
No country detected in great britain
No country detected in importe
No country detected in quebec
No country detected in quebec
No country d

No country detected in australa
No country detected in bron alizee
No country detected in kisslegg quelle krumbach naturell
No country detected in malville
No country detected in sel de guerande
No country detected in antilles francaises
No country detected in atlantique centre ouest
No country detected in polska
No country detected in pazifischer ozean fao 87 fao 77
No country detected in kakao papua neuguinea
No country detected in mazedonien
No country detected in oceano pacifico
No country detected in polska
No country detected in espanna
No country detected in sudamerica
No country detected in importe
No country detected in porc bretagne
No country detected in sud americaine
No country detected in antilles
No country detected in atlantique nord fao 27 ocean pacifique fao 67
No country detected in franca
No country detected in mediterranee
No country detected in penjab camargue
No country detected in aisne
No country detected in berry
No country detected in eu
No country detected i

No country detected in inconnu
No country detected in point vert
No country detected in au lait de normandie
No country detected in normandie
No country detected in farine de ble francais
No country detected in amerique
No country detected in import
No country detected in farine de ble francaise
No country detected in bretagne
No country detected in bretagne
No country detected in sources de soultzmatt
No country detected in cote d ivoire
No country detected in provence
No country detected in cote d ivoire
No country detected in agen region d agen aquitaine
No country detected in viande de porc union europeenne
No country detected in viande de porc union europeenne
No country detected in viande de porc union europeenne
No country detected in viande porcine union europeenne
No country detected in pays de la loire
No country detected in cote d ivoire
No country detected in cote d ivoire
No country detected in cote d ivoire
No country detected in viande de porc union europeenne
No country

No country detected in morbihan bretagne
No country detected in caraibes
No country detected in sud ouest
No country detected in vaucluse drome provencale
No country detected in amerique latine europe
No country detected in amerique latine europe
No country detected in ce hors ce
No country detected in bordeaux sarl de torrefaction
No country detected in loire
No country detected in indre et loire
No country detected in voges
No country detected in bassin de l adour aop
No country detected in pays basque
No country detected in non connue
No country detected in bretagne
No country detected in viande de porc union europeenne
No country detected in communaute europeenne
No country detected in normandie
No country detected in normandie
No country detected in baratte en normandie
No country detected in baratte en normandie
No country detected in normandie
No country detected in baratte en normandie
No country detected in creme de normandie
No country detected in normandie
No country detecte

No country detected in nicht eu landwirtschaft
No country detected in eu
No country detected in nicht eu landwirtschaft
No country detected in nordostpazifik fao 67
No country detected in wittensee
No country detected in eu landwirtschaft
No country detected in chorin
No country detected in germania
No country detected in bayern
No country detected in fao 67
No country detected in mecklenburg vorpommern
No country detected in fao 71
No country detected in deutschlan bayern ottobeuren d 87724
No country detected in republik moldau
No country detected in agpten
No country detected in nicht eu landwirtschaft
No country detected in laos
No country detected in slokawei
No country detected in nordsee
No country detected in fao 27
No country detected in eu
No country detected in eu
No country detected in alpen
No country detected in nordostpazifik
No country detected in baden wurttemberg
No country detected in baden wurttemberg
No country detected in turkei
No country detected in union europe

No country detected in zona atlantica fad27
No country detected in carnicas munoz sanchez s l tendilla guadalajara
No country detected in navarra
No country detected in california estados unidos
No country detected in turquia
No country detected in turquia
No country detected in cantabria
No country detected in espanha
No country detected in nao especificada na embalagem
No country detected in agricultura eu
No country detected in turquia
No country detected in holland
No country detected in segovia orejana
No country detected in elche alicante
No country detected in sud afrika
No country detected in cacao costa de marfil
No country detected in cantabria extremadura
No country detected in cantabria extremadura
No country detected in cuenca
No country detected in manilva malaga
No country detected in valencia
No country detected in la coruna
No country detected in leche
No country detected in turquia
No country detected in almendras estados unidos
No country detected in barcelona
No cou

In [71]:
print_random(DATA_CLEANED[DATA_CLEANED["origins_tags"].str.len() != 0 ]["origins_tags"], 10)

208194    [european union]
309193           [morocco]
228946            [france]
224712            [france]
219446            [france]
311850           [ecuador]
299955          [colombia]
308617             [china]
318686     [united-states]
307274             [spain]
Name: origins_tags, dtype: object


In [72]:
nullvalue_cleaned = len(DATA_CLEANED[DATA_CLEANED["origins_tags"].str.len() == 0])
print(f"Missing value in code after cleaning = {nullvalue_cleaned}, {nullvalue_cleaned/size*100:.2f}%")
print(f"Number of value removed = {nullvalue_cleaned - nullvalue_before}, {(nullvalue_cleaned - nullvalue_before)/size*100:.2f}%")

Missing value in code after cleaning = 300901, 93.81%
Number of value removed = 2282, 0.71%


Cette fois le nombre de lignes non traitées est plus important,  
mais il est compliqué d'aller plus loin

### Nettoyage du nitriscore
Le nutriscore doit être une lettre entre a et e  
je vérifie que la valeur renseignée est un str entre a et e en ignorant la casse

In [73]:
nullvalue_before = len(DATA[pd.isnull(DATA["nutrition_grade_fr"])])
print(f"Missing value in code before cleaning = {nullvalue_before}, {nullvalue_before/size*100:.2f}%")

Missing value in code before cleaning = 99562, 31.04%


In [74]:
DATA_CLEANED["nutrition_grade_fr"] = DATA["nutrition_grade_fr"].apply(str_in, 
                                                                      args=([['a','b','c','d','e']]), 
                                                                      ignore_case=True)

In [75]:
print_random(DATA_CLEANED[pd.notnull(DATA_CLEANED["nutrition_grade_fr"])]["nutrition_grade_fr"], 10)

97446     e
233715    d
181509    b
85983     a
232200    b
157848    c
130170    d
181063    c
244968    c
152953    a
Name: nutrition_grade_fr, dtype: object


In [76]:
nullvalue_cleaned = len(DATA_CLEANED[pd.isnull(DATA_CLEANED["nutrition_grade_fr"])])
print(f"Missing value in code after cleaning = {nullvalue_cleaned}, {nullvalue_cleaned/size*100:.2f}%")
print(f"Number of value removed = {nullvalue_cleaned - nullvalue_before}, {(nullvalue_cleaned - nullvalue_before)/size*100:.2f}%")

Missing value in code after cleaning = 99562, 31.04%
Number of value removed = 0, 0.00%


Tous les nutriscores sont correctement renseignés

### Nettoyage du nombre d'additifs
additives_n doit être un nombre entier

In [77]:
nullvalue_before = len(DATA[pd.isnull(DATA["additives_n"])])
print(f"Missing value in code before cleaning = {nullvalue_before}, {nullvalue_before/size*100:.2f}%")

Missing value in code before cleaning = 71833, 22.39%


In [78]:
DATA_CLEANED['additives_n'] = DATA['additives_n'].apply(is_int)

In [79]:
nullvalue_cleaned = len(DATA_CLEANED[pd.isnull(DATA_CLEANED["additives_n"])])
print(f"Missing value in code after cleaning = {nullvalue_cleaned}, {nullvalue_cleaned/size*100:.2f}%")
print(f"Number of value removed = {nullvalue_cleaned - nullvalue_before}, {(nullvalue_cleaned - nullvalue_before)/size*100:.2f}%")

Missing value in code after cleaning = 71833, 22.39%
Number of value removed = 0, 0.00%


Toutes les valeurs sont correctes

### Nettoyage des variables de type float :
Elles doivent être un float ou un int  
- carbon-footprint_100g
- energy_100g

NB : le nombre de valeurs non nulles est très faible pour l'empreinte carbone...

In [80]:
for variable in ["carbon-footprint_100g", "energy_100g"] :
    print('-'*50)
    print(f"Nettoyage de la variable {variable}")
    nullvalue_before = len(DATA[pd.isnull(DATA[variable])])
    print(f"Missing value in code before cleaning = {nullvalue_before}, {nullvalue_before/size*100:.2f}%")
    
    DATA_CLEANED[variable] = DATA[variable].apply(is_float)
    
    nullvalue_cleaned = len(DATA_CLEANED[pd.isnull(DATA_CLEANED[variable])])
    print(f"Missing value in code after cleaning = {nullvalue_cleaned}, {nullvalue_cleaned/size*100:.2f}%")
    print(f"Number of value removed = {nullvalue_cleaned - nullvalue_before}, {(nullvalue_cleaned - nullvalue_before)/size*100:.2f}%")

--------------------------------------------------
Nettoyage de la variable carbon-footprint_100g
Missing value in code before cleaning = 320504, 99.92%
Missing value in code after cleaning = 320504, 99.92%
Number of value removed = 0, 0.00%
--------------------------------------------------
Nettoyage de la variable energy_100g
Missing value in code before cleaning = 59659, 18.60%
Missing value in code after cleaning = 59659, 18.60%
Number of value removed = 0, 0.00%


### Nettoyage des variables représentants les apports pour 100g
les variables 'fat_100g', 'saturated-fat_100g', 'fruits-vegetables-nuts_100g', 'sodium_100g', 'sugar_100g' doivent être des floats compris entre 0 et 100.  
Si les valeurs ne sont pas dans la gamme 0-100, elles seront considérées comme abérrante et remplacées par np.NaN

In [81]:
for variable in ['fat_100g', 'saturated-fat_100g', 'fruits-vegetables-nuts_100g', 'sodium_100g', 'sugars_100g' ] :
    print('-'*50)
    print(f"Nettoyage de la variable {variable}")
    nullvalue_before = len(DATA[pd.isnull(DATA[variable])])
    print(f"Missing value in code before cleaning = {nullvalue_before}, {nullvalue_before/size*100:.2f}%")
    
    DATA_CLEANED[variable] = DATA[variable].apply(is_float)
    DATA_CLEANED[variable] = DATA_CLEANED[variable].apply(is_lower, target=100)
    DATA_CLEANED[variable] = DATA_CLEANED[variable].apply(is_higher, target=0)
    
    nullvalue_cleaned = len(DATA_CLEANED[pd.isnull(DATA_CLEANED[variable])])
    print(f"Missing value in code after cleaning = {nullvalue_cleaned}, {nullvalue_cleaned/size*100:.2f}%")
    print(f"Number of value removed = {nullvalue_cleaned - nullvalue_before}, {(nullvalue_cleaned - nullvalue_before)/size*100:.2f}%")

--------------------------------------------------
Nettoyage de la variable fat_100g
Missing value in code before cleaning = 76881, 23.97%
714.29 is higher than the target 100
380.0 is higher than the target 100
101.0 is higher than the target 100
105.0 is higher than the target 100
Missing value in code after cleaning = 76885, 23.97%
Number of value removed = 4, 0.00%
--------------------------------------------------
Nettoyage de la variable saturated-fat_100g
Missing value in code before cleaning = 91218, 28.44%
175.38 is higher than the target 100
210.0 is higher than the target 100
550.0 is higher than the target 100
Missing value in code after cleaning = 91221, 28.44%
Number of value removed = 3, 0.00%
--------------------------------------------------
Nettoyage de la variable fruits-vegetables-nuts_100g
Missing value in code before cleaning = 317736, 99.05%
Missing value in code after cleaning = 317736, 99.05%
Number of value removed = 0, 0.00%
----------------------------------

### Nettoyage catégories pnns
les valeurs doivent être de type str,  
les valeurs unknown sont remplacées par np.NaN  
Les espaces sont remplacés par des '_'
le texte est passé en minuscules uniquement
Il s'avère qu'initialement des '-' peuvent se placer à la place des espaces, suppression.

In [82]:
nullvalue_before = len(DATA[pd.isnull(DATA["pnns_groups_1"])])
print(f"Missing value in code before cleaning = {nullvalue_before}, {nullvalue_before/size*100:.2f}%")

Missing value in code before cleaning = 229259, 71.47%


In [83]:
DATA_CLEANED["pnns_groups_1"] = DATA["pnns_groups_1"].apply(is_str).apply(lower_case).apply(str_replace, args=("-", " ")).apply(str_replace, args=(" ", "_")).apply(remove_str, args=("unknown",))

In [84]:
print_random(DATA_CLEANED[pd.notnull(DATA_CLEANED["pnns_groups_1"])]["pnns_groups_1"], 10)

174898       cereals_and_potatoes
216215    milk_and_dairy_products
262114              sugary_snacks
205367    milk_and_dairy_products
196788              sugary_snacks
233505             fish_meat_eggs
255277                  beverages
282889       cereals_and_potatoes
225018             fish_meat_eggs
178170    milk_and_dairy_products
Name: pnns_groups_1, dtype: object


In [85]:
nullvalue_cleaned = len(DATA_CLEANED[pd.isnull(DATA_CLEANED["pnns_groups_1"])])
print(f"Missing value in code after cleaning = {nullvalue_cleaned}, {nullvalue_cleaned/size*100:.2f}%")
print(f"Number of value removed = {nullvalue_cleaned - nullvalue_before}, {(nullvalue_cleaned - nullvalue_before)/size*100:.2f}%")

Missing value in code after cleaning = 251883, 78.52%
Number of value removed = 22624, 7.05%


In [86]:
nullvalue_before = len(DATA[pd.isnull(DATA["pnns_groups_2"])])
print(f"Missing value in code before cleaning = {nullvalue_before}, {nullvalue_before/size*100:.2f}%")

Missing value in code before cleaning = 226281, 70.54%


In [87]:
DATA_CLEANED["pnns_groups_2"] = DATA["pnns_groups_2"].apply(is_str).apply(lower_case).apply(str_replace, args=("-", " ")).apply(str_replace, args=(" ", "_")).apply(remove_str, args=("unknown",))

In [88]:
print_random(DATA_CLEANED[pd.notnull(DATA_CLEANED["pnns_groups_2"])]["pnns_groups_2"], 10)

311299               vegetables
219089                ice_cream
237493             fruit_juices
218358           one_dish_meals
252031     dressings_and_sauces
240674                   sweets
270065                  cereals
199364      alcoholic_beverages
264540                  legumes
309562    non_sugared_beverages
Name: pnns_groups_2, dtype: object


In [89]:
nullvalue_cleaned = len(DATA_CLEANED[pd.isnull(DATA_CLEANED["pnns_groups_2"])])
print(f"Missing value in code after cleaning = {nullvalue_cleaned}, {nullvalue_cleaned/size*100:.2f}%")
print(f"Number of value removed = {nullvalue_cleaned - nullvalue_before}, {(nullvalue_cleaned - nullvalue_before)/size*100:.2f}%")

Missing value in code after cleaning = 248905, 77.60%
Number of value removed = 22624, 7.05%


Un nombre important de valeurs ont été supprimées, correspondant aux valeurs unknown

### Détection des doublons

In [90]:
effectifs_code = DATA_CLEANED["code"].value_counts()
print(f"Nombre de doublons détectés à partir du code : {len(effectifs_code[effectifs_code > 1])}")

Nombre de doublons détectés à partir du code : 0


### Suppression des doublons

je crée volontairement un doublon pour le développement des fonctions

In [91]:
#dup = DATA_CLEANED[DATA_CLEANED["code"] == "8712100872044"]
#DATA_DUP = pd.concat([DATA_CLEANED, dup], axis=0, ignore_index=True)
#effectifs_code = DATA_DUP["code"].value_counts()

In [92]:
#print(len(DATA_DUP))
#for code in effectifs_code[effectifs_code > 1].index :
#    print(code)
#    duplicates = DATA_DUP[DATA_DUP["code"] == code ]
#    DATA_DUP.drop(DATA_DUP[DATA_DUP["code"] == code].index, inplace=True)
#    #print(len(DATA_DUP))
#    #print(duplicates)
#    new_rows = merge_rows(duplicates)
#    print(new_rows)

In [93]:
effectifs_code = DATA_CLEANED["code"].value_counts()
for code in effectifs_code[effectifs_code > 1].index :
    print(f"{code} is duplicated, merge all rows")
    duplicates = DATA_CLEANED[DATA_CLEANED["code"] == code ]
    DATA_CLEANED.drop(DATA_CLEANED[DATA_CLEANED["code"] == code].index, inplace=True)
    new_row = merge_rows(duplicates)
    DATA_CLEANED = pd.concat([DATA_CLEANED, new_row], axis=0, ignore_index=True)

### Conversion des variables de type list en str 
Pour la sauvegarde il est préférable de repasser les varibales de type list en str

In [94]:
for col in ['categories_tags', 'origins_tags', 'manufacturing_places_tags'] :
    DATA_CLEANED[col] = DATA_CLEANED[col].apply(list2str)

### Sauvegarde de la base de données nettoyer
sauvegarde au format csv délimité par ";"  
et au format pickle

In [95]:
DATA_CLEANED.to_csv("database_cleaned.csv", sep=";", index=False)
with open("database_cleaned.pkl", "wb") as ofile :
    pickle.dump(DATA_CLEANED, ofile)